## Project - Airline AI Assistant (using tools/function calling)
An AI Customer Support assistant for an Airline which will answer queries and if it is asked about flight fare, it will call a function to check for the fare and then respond with the correct flight fare and also create and display a picture of the destination city.

In [ ]:
# Imports

from openai import OpenAI
import gradio as gr
import json
from dotenv import load_dotenv
import os
import base64
from io import BytesIO
from PIL import Image

In [ ]:
# Initializing

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
# Prompt

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence."
system_message += "Always be accurate. If you don't know the answer to any question, say so."

### Tool

In [ ]:
# Flight fares
ticket_prices = {"kolkata": "₹5500", "bengaluru": "₹8500", "delhi": "₹7000", "mumbai": "₹9499"}

# Function to get flight fares from the above dict.
def get_ticket_price(destination_city):
    print(f"Tool: get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

get_ticket_price("Berlin")

Tool: get_ticket_price called for Berlin


'Unknown'

In [6]:
get_ticket_price("Mumbai")

Tool: get_ticket_price called for Mumbai


'₹9499'

In [ ]:
# Structure to describe our function/tool:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price,\
                    for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# Adding in a list of tools:

tools = [{"type": "function", "function": price_function}]

### Getting OpenAI to use our Tool

In [ ]:
# We have written function to handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
# Function to draw the destination city in a pop-art style to be displayed in the UI.

def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
# Creating our char interface.

def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    
    return history, image

In [ ]:
# Creating UI for our chat-interface.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


message: hi
response: ChatCompletion(id='chatcmpl-ByBiibD2XUpxw3PJYinUDzVKvS8Kx', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1753684432, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_62a23a81ef', usage=CompletionUsage(completion_tokens=10, prompt_tokens=133, total_tokens=143, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
message: what can you do?
response: ChatCompletion(id='chatcmpl-ByBiqgkJXDqmLZFB5kwnZnlICmIuT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I can help you with flight information, 